<a href="https://colab.research.google.com/github/seonghaiiiii/jeju-food/blob/main/autogluon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import os

from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
!pip install -U autogluon

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/base_command.py", line 169, in exc_logging_wrapper
    status = run_func(*args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/cli/req_command.py", line 242, in wrapper
    return func(self, options, args)
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/commands/install.py", line 377, in run
    requirement_set = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_internal/resolution/resolvelib/resolver.py", line 92, in resolve
    result = self._result = resolver.resolve(
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 546, in resolve
    state = resolution.resolve(requirements, max_rounds=max_rounds)
  File "/usr/local/lib/python3.10/dist-packages/pip/_vendor/resolvelib/resolvers.py", line 426, in resolve
    name = min(unsatisfied_names, key=self._get_preference)
  File "/usr/local/lib/python3.

In [ ]:
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# data load
train = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/train.csv')
trade = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/international_trade.csv')
test = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/test.csv')
df1 = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/sample_submission.csv')

In [ ]:
# ID 열의 앞 6자를 기준으로 groupby하여 데이터프레임을 분할
grouped = train.groupby(train['ID'].str[:6])

# 그룹별로 데이터프레임을 만들어 저장
dataframes = {}
for group_name, group_data in grouped:
    dataframes[group_name] = group_data

# 결과 확인
for group_name, group_df in dataframes.items():
    print(f"Group: {group_name}")
    print(group_df)


Group: BC_A_J
                    ID   timestamp item corporation location  supply(kg)  \
42644  BC_A_J_20190101  2019-01-01   BC           A        J         0.0   
42645  BC_A_J_20190102  2019-01-02   BC           A        J         0.0   
42646  BC_A_J_20190103  2019-01-03   BC           A        J      7616.0   
42647  BC_A_J_20190104  2019-01-04   BC           A        J      7488.0   
42648  BC_A_J_20190105  2019-01-05   BC           A        J     10408.0   
...                ...         ...  ...         ...      ...         ...   
44162  BC_A_J_20230227  2023-02-27   BC           A        J     11702.0   
44163  BC_A_J_20230228  2023-02-28   BC           A        J      6056.0   
44164  BC_A_J_20230301  2023-03-01   BC           A        J      5896.0   
44165  BC_A_J_20230302  2023-03-02   BC           A        J      8600.0   
44166  BC_A_J_20230303  2023-03-03   BC           A        J     10728.0   

       price(원/kg)  
42644          0.0  
42645          0.0  
42646     

In [ ]:
# 'ID' 열의 고유한 값 확인
unique_ids = df1['ID'].unique()

# 고유한 값들 출력
for unique_id in unique_ids:
    print(unique_id)

TG_A_J_20230304
TG_A_J_20230305
TG_A_J_20230306
TG_A_J_20230307
TG_A_J_20230308
TG_A_J_20230309
TG_A_J_20230310
TG_A_J_20230311
TG_A_J_20230312
TG_A_J_20230313
TG_A_J_20230314
TG_A_J_20230315
TG_A_J_20230316
TG_A_J_20230317
TG_A_J_20230318
TG_A_J_20230319
TG_A_J_20230320
TG_A_J_20230321
TG_A_J_20230322
TG_A_J_20230323
TG_A_J_20230324
TG_A_J_20230325
TG_A_J_20230326
TG_A_J_20230327
TG_A_J_20230328
TG_A_J_20230329
TG_A_J_20230330
TG_A_J_20230331
TG_A_S_20230304
TG_A_S_20230305
TG_A_S_20230306
TG_A_S_20230307
TG_A_S_20230308
TG_A_S_20230309
TG_A_S_20230310
TG_A_S_20230311
TG_A_S_20230312
TG_A_S_20230313
TG_A_S_20230314
TG_A_S_20230315
TG_A_S_20230316
TG_A_S_20230317
TG_A_S_20230318
TG_A_S_20230319
TG_A_S_20230320
TG_A_S_20230321
TG_A_S_20230322
TG_A_S_20230323
TG_A_S_20230324
TG_A_S_20230325
TG_A_S_20230326
TG_A_S_20230327
TG_A_S_20230328
TG_A_S_20230329
TG_A_S_20230330
TG_A_S_20230331
TG_B_J_20230304
TG_B_J_20230305
TG_B_J_20230306
TG_B_J_20230307
TG_B_J_20230308
TG_B_J_20230309
TG_B_J_2

In [ ]:
#ID 앞에 6자만 필요해서 자른거임
df1['ID_1'] = df1['ID'].str[:6]

In [ ]:
# 'ID' 열의 고유한 값 확인
unique_ids = df1['ID_1'].unique()

unique_ids

# 총 39개

array(['TG_A_J', 'TG_A_S', 'TG_B_J', 'TG_B_S', 'TG_C_J', 'TG_C_S',
       'TG_D_J', 'TG_D_S', 'TG_E_J', 'TG_E_S', 'CR_A_J', 'CR_B_J',
       'CR_C_J', 'CR_D_J', 'CR_D_S', 'CR_E_J', 'CR_E_S', 'CB_A_J',
       'CB_A_S', 'CB_D_J', 'CB_E_J', 'RD_A_J', 'RD_A_S', 'RD_C_S',
       'RD_D_J', 'RD_D_S', 'RD_E_J', 'RD_E_S', 'BC_A_J', 'BC_A_S',
       'BC_B_J', 'BC_B_S', 'BC_C_J', 'BC_C_S', 'BC_D_J', 'BC_E_J',
       'BC_E_S', 'CB_F_J', 'RD_F_J'], dtype=object)

In [ ]:
TG_A_J = dataframes.get('TG_A_J')
TG_A_S = dataframes.get('TG_A_S')
TG_B_J = dataframes.get('TG_B_J')
TG_B_S = dataframes.get('TG_B_S')
TG_C_J = dataframes.get('TG_C_J')
TG_C_S = dataframes.get('TG_C_S')
TG_D_J = dataframes.get('TG_D_J')
TG_D_S = dataframes.get('TG_D_S')
TG_E_J = dataframes.get('TG_E_J')
TG_E_S = dataframes.get('TG_E_S')
CR_A_J = dataframes.get('CR_A_J')
CR_B_J = dataframes.get('CR_B_J')
CR_C_J = dataframes.get('CR_C_J')
CR_D_J = dataframes.get('CR_D_J')
CR_D_S = dataframes.get('CR_D_S')
CR_E_J = dataframes.get('CR_E_J')
CR_E_S = dataframes.get('CR_E_S')
CB_A_J = dataframes.get('CB_A_J')
CB_A_S = dataframes.get('CB_A_S')
CB_D_J = dataframes.get('CB_D_J')
CB_E_J = dataframes.get('CB_E_J')
RD_A_J = dataframes.get('RD_A_J')
RD_A_S = dataframes.get('RD_A_S')
RD_C_S = dataframes.get('RD_C_S')
RD_D_J = dataframes.get('RD_D_J')
RD_D_S = dataframes.get('RD_D_S')
RD_E_J = dataframes.get('RD_E_J')
RD_E_S = dataframes.get('RD_E_S')
BC_A_J = dataframes.get('BC_A_J')
BC_A_S = dataframes.get('BC_A_S')
BC_B_J = dataframes.get('BC_B_J')
BC_B_S = dataframes.get('BC_B_S')
BC_C_J = dataframes.get('BC_C_J')
BC_C_S = dataframes.get('BC_C_S')
BC_D_J = dataframes.get('BC_D_J')
BC_E_J = dataframes.get('BC_E_J')
BC_E_S = dataframes.get('BC_E_S')
CB_F_J = dataframes.get('CB_F_J')
RD_F_J = dataframes.get('RD_F_J')

In [ ]:
TG_A_J

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
1518,TG_A_J_20230227,2023-02-27,TG,A,J,7009.6,2657.0
1519,TG_A_J_20230228,2023-02-28,TG,A,J,6775.0,3922.0
1520,TG_A_J_20230301,2023-03-01,TG,A,J,7696.7,3397.0
1521,TG_A_J_20230302,2023-03-02,TG,A,J,5815.0,3195.0


In [ ]:
# 삭제할 열 이름 리스트
columns_to_delete = ['item', 'corporation', 'location','supply(kg)']
# 변수들에 대해 열 삭제 수행
dataframes = [TG_A_J, TG_A_S, TG_B_J, TG_B_S, TG_C_J, TG_C_S, TG_D_J, TG_D_S, TG_E_J, TG_E_S,
              CR_A_J, CR_B_J, CR_C_J, CR_D_J, CR_D_S, CR_E_J, CR_E_S,
              CB_A_J, CB_A_S, CB_D_J, CB_E_J,
              RD_A_J, RD_A_S, RD_C_S, RD_D_J, RD_D_S, RD_E_J, RD_E_S,
              BC_A_J, BC_A_S, BC_B_J, BC_B_S, BC_C_J, BC_C_S, BC_D_J, BC_E_J, BC_E_S,
              CB_F_J, RD_F_J]

for df in dataframes:
    df.drop(columns=columns_to_delete, inplace=True)


In [ ]:
import numpy as np

def calculate_surrounding_mean_price(index, data):
    # Ensure we don't go out of bounds
    start_index = max(index - 1, 0)
    end_index = min(index + 1, len(data))

    # Select the surrounding days, excluding the current day
    surrounding_prices = data.iloc[start_index:index].append(data.iloc[index+1:end_index])

    # Calculate the mean, excluding zero prices
    mean_price = surrounding_prices[surrounding_prices != 0].mean()

    # If there are no non-zero prices, return 0
    if np.isnan(mean_price):
        return 0
    else:
        return mean_price

group_names = ["TG_A_J", "TG_A_S", "TG_B_J", "TG_B_S", "TG_C_J", "TG_C_S", "TG_D_J", "TG_D_S", "TG_E_J", "TG_E_S",
               "CR_A_J", "CR_B_J", "CR_C_J", "CR_D_J", "CR_D_S", "CR_E_J", "CR_E_S",
               "CB_A_J", "CB_A_S", "CB_D_J", "CB_E_J",
               "RD_A_J", "RD_A_S", "RD_C_S", "RD_D_J", "RD_D_S", "RD_E_J", "RD_E_S",
               "BC_A_J", "BC_A_S", "BC_B_J", "BC_B_S", "BC_C_J", "BC_C_S", "BC_D_J", "BC_E_J", "BC_E_S",
               "CB_F_J", "RD_F_J"]

# Apply the function to each dataframe in the list
for group_df in dataframes:
    zero_price_indices = group_df[group_df['price(원/kg)'] == 0].index
    for index in zero_price_indices:
        group_df.at[index, 'price(원/kg)'] = calculate_surrounding_mean_price(index, group_df['price(원/kg)'])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
<ipython-input-13-b0f5bfeec851>:9: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  surrounding_prices = data.iloc[start_index:index].append(data.iloc[index+1:end_index])
<ipython-input-13-b0f5bfeec851>:9: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  surrounding_prices = data.iloc[start_index:index].append(data.iloc[index+1:end_index])
<ipython-input-13-b0f5bfeec851>:9: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  surrounding_prices = data.iloc[start_index:index].append(data.iloc[index+1:end_index])
<ipython-input-13-b0f5bfeec851>:9: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  surrounding_prices 

In [ ]:
TG_A_S.head(50)

,ID,timestamp,price(원/kg)
1523,TG_A_S_20190101,2019-01-01,4089.0
1524,TG_A_S_20190102,2019-01-02,0.0
1525,TG_A_S_20190103,2019-01-03,2526.0
1526,TG_A_S_20190104,2019-01-04,2134.0
1527,TG_A_S_20190105,2019-01-05,2075.0
1528,TG_A_S_20190106,2019-01-06,0.0
1529,TG_A_S_20190107,2019-01-07,2181.0
1530,TG_A_S_20190108,2019-01-08,2029.0
1531,TG_A_S_20190109,2019-01-09,1998.0
1532,TG_A_S_20190110,2019-01-10,2023.0


In [ ]:
train.head(50)

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
5,TG_A_J_20190106,2019-01-06,TG,A,J,0.0,0.0
6,TG_A_J_20190107,2019-01-07,TG,A,J,44995.0,1474.0
7,TG_A_J_20190108,2019-01-08,TG,A,J,26975.0,1326.0
8,TG_A_J_20190109,2019-01-09,TG,A,J,29265.0,1428.0
9,TG_A_J_20190110,2019-01-10,TG,A,J,21226.0,1433.0


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/train.csv')

In [ ]:
# TG_A_J의 'price(원/kg)'을 train과 'ID'를 기준으로 업데이트
train = pd.merge(train, TG_A_J[['ID', 'price(원/kg)']], on='ID', how='left', suffixes=('_train', '_TG_A_J'))
train['price(원/kg)_train'] = train['price(원/kg)_TG_A_J'].combine_first(train['price(원/kg)_train'])
train.drop('price(원/kg)_TG_A_J', axis=1, inplace=True)
train.rename(columns={'price(원/kg)_train': 'price(원/kg)'}, inplace=True)
# 결과 확인
train.head(10)


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
5,TG_A_J_20190106,2019-01-06,TG,A,J,0.0,1250.0
6,TG_A_J_20190107,2019-01-07,TG,A,J,44995.0,1474.0
7,TG_A_J_20190108,2019-01-08,TG,A,J,26975.0,1326.0
8,TG_A_J_20190109,2019-01-09,TG,A,J,29265.0,1428.0
9,TG_A_J_20190110,2019-01-10,TG,A,J,21226.0,1433.0


In [ ]:
group_names = [TG_A_J, TG_A_S, TG_B_J, TG_B_S, TG_C_J, TG_C_S, TG_D_J, TG_D_S, TG_E_J, TG_E_S,
               CR_A_J, CR_B_J, CR_C_J, CR_D_J, CR_D_S, CR_E_J, CR_E_S,
               CB_A_J, CB_A_S, CB_D_J, CB_E_J,
               RD_A_J, RD_A_S, RD_C_S, RD_D_J, RD_D_S, RD_E_J, RD_E_S,
               BC_A_J, BC_A_S, BC_B_J, BC_B_S, BC_C_J, BC_C_S, BC_D_J, BC_E_J, BC_E_S,
               CB_F_J, RD_F_J]
for current_dataframe in group_names:

    # train과 ID가 같으면 train의 price(원/kg)을 각 데이터프레임의 price(원/kg)로 바꾸기
    train = pd.merge(train, current_dataframe[['ID', 'price(원/kg)']], on='ID', how='left', suffixes=('_train', f'_{current_dataframe}'))
    train[f'price(원/kg)_train'] = train[f'price(원/kg)_{current_dataframe}'].combine_first(train['price(원/kg)_train'])
    train.drop(f'price(원/kg)_{current_dataframe}', axis=1, inplace=True)
    train.rename(columns={'price(원/kg)_train': 'price(원/kg)'}, inplace=True)

# 최종 결과 확인
print(train.head(10))

                ID   timestamp item corporation location  supply(kg)  \
0  TG_A_J_20190101  2019-01-01   TG           A        J         0.0   
1  TG_A_J_20190102  2019-01-02   TG           A        J         0.0   
2  TG_A_J_20190103  2019-01-03   TG           A        J     60601.0   
3  TG_A_J_20190104  2019-01-04   TG           A        J     25000.0   
4  TG_A_J_20190105  2019-01-05   TG           A        J     32352.0   
5  TG_A_J_20190106  2019-01-06   TG           A        J         0.0   
6  TG_A_J_20190107  2019-01-07   TG           A        J     44995.0   
7  TG_A_J_20190108  2019-01-08   TG           A        J     26975.0   
8  TG_A_J_20190109  2019-01-09   TG           A        J     29265.0   
9  TG_A_J_20190110  2019-01-10   TG           A        J     21226.0   

   price(원/kg)  
0          0.0  
1          0.0  
2       1728.0  
3       1408.0  
4       1250.0  
5       1250.0  
6       1474.0  
7       1326.0  
8       1428.0  
9       1433.0  


In [ ]:
train


,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


#autogluon


In [ ]:
train_df = train[['ID','timestamp','supply(kg)', 'price(원/kg)']]
# test_df = pd.read_csv('test.csv')[['ID','timestamp']]

In [ ]:
train_df['item_id'] = train_df.ID.str[0:6]
# test_df['item_id'] = test_df.ID.str[0:6]

<ipython-input-11-a9e14e50f493>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['item_id'] = train_df.ID.str[0:6]


In [ ]:
train_df

,ID,timestamp,supply(kg),price(원/kg),item_id
0,TG_A_J_20190101,2019-01-01,0.0,0.0,TG_A_J
1,TG_A_J_20190102,2019-01-02,0.0,0.0,TG_A_J
2,TG_A_J_20190103,2019-01-03,60601.0,1728.0,TG_A_J
3,TG_A_J_20190104,2019-01-04,25000.0,1408.0,TG_A_J
4,TG_A_J_20190105,2019-01-05,32352.0,1250.0,TG_A_J
...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,452440.0,468.0,RD_F_J
59393,RD_F_J_20230228,2023-02-28,421980.0,531.0,RD_F_J
59394,RD_F_J_20230301,2023-03-01,382980.0,574.0,RD_F_J
59395,RD_F_J_20230302,2023-03-02,477220.0,523.0,RD_F_J


In [ ]:
trade = trade[trade['품목명'].isin(['감귤', '당근', '양배추', '꽃양배추와 브로콜리'])]

In [ ]:
fruits_dict = {'감귤':'TG' ,'꽃양배추와 브로콜리':'BC' ,'당근':'CR' ,'양배추':'CB'}
trade['품목명'] = trade['품목명'].map(fruits_dict)

In [ ]:
trade = trade[['기간', '품목명', '무역수지']]

In [ ]:
trade['기간'] = pd.to_datetime(trade['기간'])

# 테스트 데이터에 대해 동일한 작업 수행
trade['Year'] = trade['기간'].dt.year
trade['Month'] = trade['기간'].dt.month

In [ ]:
trade

,기간,품목명,무역수지,Year,Month
5,2019-01-01,CB,4,2019,1
8,2019-01-01,CR,-2934,2019,1
17,2019-01-01,TG,172,2019,1
30,2019-02-01,CB,-8,2019,2
33,2019-02-01,CR,-2585,2019,2
...,...,...,...,...,...
1226,2023-01-01,CR,-2321,2023,1
1236,2023-01-01,TG,269,2023,1
1250,2023-02-01,CB,-91,2023,2
1253,2023-02-01,CR,-3737,2023,2


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/train.csv')

In [ ]:
train['timestamp'] = pd.to_datetime(train['timestamp'])

# 테스트 데이터에 대해 동일한 작업 수행
train['Year'] = train['timestamp'].dt.year
train['Month'] = train['timestamp'].dt.month

In [ ]:
train = train[['ID','timestamp','supply(kg)', 'price(원/kg)','item','Year','Month']]

In [ ]:
trade.rename(columns={'품목명': 'item'}, inplace=True)

In [ ]:
# 데이터프레임 병합
merged_data = pd.merge(train, trade, how='left', on=['Year', 'Month', 'item'])

# 필요한 열만 선택하고 NaN을 0으로 대체
merged_data['무역수지'].fillna(0, inplace=True)

# tradenum 열에 무역수지 대입
merged_data['tradenum'] = merged_data['무역수지']

merged_data

,ID,timestamp,supply(kg),price(원/kg),item,Year,Month,기간,무역수지,tradenum
0,TG_A_J_20190101,2019-01-01,0.0,0.0,TG,2019,1,2019-01-01,172.0,172.0
1,TG_A_J_20190102,2019-01-02,0.0,0.0,TG,2019,1,2019-01-01,172.0,172.0
2,TG_A_J_20190103,2019-01-03,60601.0,1728.0,TG,2019,1,2019-01-01,172.0,172.0
3,TG_A_J_20190104,2019-01-04,25000.0,1408.0,TG,2019,1,2019-01-01,172.0,172.0
4,TG_A_J_20190105,2019-01-05,32352.0,1250.0,TG,2019,1,2019-01-01,172.0,172.0
...,...,...,...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,452440.0,468.0,RD,2023,2,NaT,0.0,0.0
59393,RD_F_J_20230228,2023-02-28,421980.0,531.0,RD,2023,2,NaT,0.0,0.0
59394,RD_F_J_20230301,2023-03-01,382980.0,574.0,RD,2023,3,NaT,0.0,0.0
59395,RD_F_J_20230302,2023-03-02,477220.0,523.0,RD,2023,3,NaT,0.0,0.0


In [ ]:
# Year, Month, Day, 기간, 무역수지 열 삭제
merged_data = merged_data.drop(['Year', 'Month', '기간', '무역수지','item'], axis=1)

# 결과 확인
print(merged_data)


                    ID  timestamp  supply(kg)  price(원/kg)  tradenum
0      TG_A_J_20190101 2019-01-01         0.0          0.0     172.0
1      TG_A_J_20190102 2019-01-02         0.0          0.0     172.0
2      TG_A_J_20190103 2019-01-03     60601.0       1728.0     172.0
3      TG_A_J_20190104 2019-01-04     25000.0       1408.0     172.0
4      TG_A_J_20190105 2019-01-05     32352.0       1250.0     172.0
...                ...        ...         ...          ...       ...
59392  RD_F_J_20230227 2023-02-27    452440.0        468.0       0.0
59393  RD_F_J_20230228 2023-02-28    421980.0        531.0       0.0
59394  RD_F_J_20230301 2023-03-01    382980.0        574.0       0.0
59395  RD_F_J_20230302 2023-03-02    477220.0        523.0       0.0
59396  RD_F_J_20230303 2023-03-03    427520.0        529.0       0.0

[59397 rows x 5 columns]


In [ ]:
train = merged_data

In [ ]:
train

,ID,timestamp,supply(kg),price(원/kg),tradenum
0,TG_A_J_20190101,2019-01-01,0.0,0.0,172.0
1,TG_A_J_20190102,2019-01-02,0.0,0.0,172.0
2,TG_A_J_20190103,2019-01-03,60601.0,1728.0,172.0
3,TG_A_J_20190104,2019-01-04,25000.0,1408.0,172.0
4,TG_A_J_20190105,2019-01-05,32352.0,1250.0,172.0
...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,452440.0,468.0,0.0
59393,RD_F_J_20230228,2023-02-28,421980.0,531.0,0.0
59394,RD_F_J_20230301,2023-03-01,382980.0,574.0,0.0
59395,RD_F_J_20230302,2023-03-02,477220.0,523.0,0.0


In [ ]:
train['item_id'] = train.ID.str[0:6]

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 59397 entries, 0 to 59396
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   ID           59397 non-null  object        
 1   timestamp    59397 non-null  datetime64[ns]
 2   supply(kg)   59397 non-null  float64       
 3   price(원/kg)  59397 non-null  float64       
 4   tradenum     59397 non-null  float64       
 5   item_id      59397 non-null  object        
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 3.2+ MB


In [ ]:
data = TimeSeriesDataFrame(train.drop(columns=['ID']))
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",
)
# seed 고정
predictor.fit( data, random_seed=42, )


No path specified. Models will be saved in: "AutogluonModels/ag-20231113_081114/"
================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231113_081114/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains foll

In [ ]:
from autogluon import TimeSeriesPredictor as task
from autogluon.tabular import TabularDataset

# 데이터프레임 생성
data = TabularDataset(train.drop(columns=['ID']))

# TimeSeriesPredictor 초기화 및 훈련
context_length = 1500
predictor = task(label="price(원/kg)", eval_metric="rmse", hyperparameters={'context_length': context_length}).fit(train_data=data, time_limits=600)


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ /usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3553 in run_code        │
│                                                                                                  │
│   3550 │   │   │   │   elif async_ :                                                             │
│   3551 │   │   │   │   │   await eval(code_obj, self.user_global_ns, self.user_ns)               │
│   3552 │   │   │   │   else:                                                                     │
│ ❱ 3553 │   │   │   │   │   exec(code_obj, self.user_global_ns, self.user_ns)                     │
│   3554 │   │   │   finally:                                                                      │
│   3555 │   │   │   │   # Reset our crash handler in place                                        │
│   3556 │   │   │   │   sys.excepthook = old_excepthook                                           │
│ in <cell line: 1>:1                                                                              │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ImportError: cannot import name 'TimeSeriesPredictor' from 'autogluon' (unknown location)

In [ ]:
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	7.13    s     = Training runtime
Fitting model: DeepAR_FULL | Skipping fit via cloning parent ...
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['Naive_FULL', 'SeasonalNaive_FULL', 'Theta_FULL', 'AutoETS_FULL', 'RecursiveTabular_FULL', 'DeepAR_FULL', 'WeightedEnsemble_FULL']
Total runtime: 7.21 s
Updated best model to 'WeightedEnsemble_FULL' (Previously 'W

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR': 'DeepAR_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [ ]:
# seed 고정
pred = predictor.predict(data, random_seed=42, )

INFO:lightning_fabric.utilities.seed:Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [ ]:
pred

mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3267.430632  1331.178563  2009.883061  2500.718941   
        2023-03-05   613.399876 -1640.499803  -868.076011  -311.984476   
        2023-03-06  3062.702166  -297.482752  1020.896791  1827.865113   
        2023-03-07  3457.144647  -256.408123  1014.433332  2004.326054   
        2023-03-08  3357.136627  -540.531784   843.662040  1801.150393   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   492.997841  -154.064923   100.713694   261.549080   
        2023-03-28   516.488705  -186.772902   102.598754   258.189246   
        2023-03-29   514.444653  -196.348553    96.625750   258.580808   
        2023-03-30   491.955314  -222.693186    80.035492   242.278303   
        2023-03-31   477.655817  -283.472906    60.371396   230.521447   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  2917.926294  3289.385218  3666.367876  4057.338970   
        2023-03-05   157.961232   593.611114  1041.550743  1515.186085   
        2023-03-06  2472.375984  3070.894940  3670.766445  4346.067951   
        2023-03-07  2747.181574  3448.386784  4150.071801  4876.925690   
        2023-03-08  2611.578797  3370.875726  4137.962038  4940.406982   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   393.129356   512.939985   631.683710   759.471722   
        2023-03-28   398.162304   523.497273   645.413322   781.181480   
        2023-03-29   395.465607   523.376401   647.572299   782.616567   
        2023-03-30   376.859391   499.855953   623.765194   762.937967   
        2023-03-31   367.150515   492.715453   625.913751   762.547067   

                            0.8          0.9  
item_id timestamp                             
TG_A_J  2023-03-04  4544.137411  5317.154420  
        2023-03-05  2068.962429  2833.426812  
        2023-03-06  5110.855335  6369.115225  
        2023-03-07  5732.017292  7090.121087  
        2023-03-08  5946.660945  7311.684927  
...                         ...          ...  
RD_F_J  2023-03-27   909.018135  1128.595079  
        2023-03-28   938.677191  1165.003650  
        2023-03-29   941.959033  1163.596652  
        2023-03-30   925.997067  1149.978445  
        2023-03-31   929.231264  1144.579621  

[1092 rows x 10 columns]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/sample_submission.csv')

In [ ]:
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
submission

,ID,answer
0,TG_A_J_20230304,3267.430632
1,TG_A_J_20230305,613.399876
2,TG_A_J_20230306,3062.702166
3,TG_A_J_20230307,3457.144647
4,TG_A_J_20230308,3357.136627
...,...,...
1087,RD_F_J_20230327,492.997841
1088,RD_F_J_20230328,516.488705
1089,RD_F_J_20230329,514.444653
1090,RD_F_J_20230330,491.955314


In [ ]:
submission.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1092 entries, 0 to 1091
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      1092 non-null   object 
 1   answer  1092 non-null   float64
dtypes: float64(1), object(1)
memory usage: 17.2+ KB


In [ ]:
from datetime import datetime

# 'ID' 열에서 뒤 8글자를 추출하여 'date' 열을 만듭니다.
submission['date'] = pd.to_datetime(submission['ID'].str[-8:], format='%Y%m%d')

# 'date' 열에서 요일을 추출하여 'day_of_week' 열을 만듭니다. (0: 월요일, 1: 화요일, ..., 6: 일요일)
submission['day_of_week'] = submission['date'].dt.dayofweek

# 'day_of_week'이 6인 (일요일) 행들의 'answer' 값을 0으로 설정합니다.
submission.loc[submission['day_of_week'] == 6, 'answer'] = 0

# 'date' 및 'day_of_week' 열은 더 이상 필요하지 않으므로 제거합니다.
submission = submission.drop(['date', 'day_of_week'], axis=1)


In [ ]:
submission

,ID,answer
0,TG_A_J_20230304,3267.430632
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3062.702166
3,TG_A_J_20230307,3457.144647
4,TG_A_J_20230308,3357.136627
...,...,...
1087,RD_F_J_20230327,492.997841
1088,RD_F_J_20230328,516.488705
1089,RD_F_J_20230329,514.444653
1090,RD_F_J_20230330,491.955314


In [ ]:
submission.to_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/autogluontrade.csv', index=False)

#다시

In [ ]:
train_df = train[['ID','timestamp','supply(kg)', 'price(원/kg)']]
# test_df = pd.read_csv('test.csv')[['ID','timestamp']]

In [ ]:
train_df['item_id'] = train_df.ID.str[0:6]
# test_df['item_id'] = test_df.ID.str[0:6]

<ipython-input-35-a9e14e50f493>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train_df['item_id'] = train_df.ID.str[0:6]


In [ ]:
data = TimeSeriesDataFrame(train_df.drop(columns=['ID']))
predictor = TimeSeriesPredictor(
    prediction_length=28,
    target="price(원/kg)",
    eval_metric="RMSE",
)
# seed 고정
predictor.fit( data, random_seed=42, )

================ TimeSeriesPredictor ================
TimeSeriesPredictor.fit() called
Fitting with arguments:
{'enable_ensemble': True,
 'evaluation_metric': 'RMSE',
 'excluded_model_types': None,
 'hyperparameter_tune_kwargs': None,
 'hyperparameters': 'default',
 'num_val_windows': 1,
 'prediction_length': 28,
 'random_seed': 42,
 'target': 'price(원/kg)',
 'time_limit': None,
 'verbosity': 2}
Provided training data set with 59397 rows, 39 items (item = single time series). Average time series length is 1523.0. Data frequency is 'D'.
INFO:lightning_fabric.utilities.seed:Global seed set to 42
AutoGluon will save models to AutogluonModels/ag-20231113_143231/
AutoGluon will gauge predictive performance using evaluation metric: 'RMSE'
	This metric's sign has been flipped to adhere to being 'higher is better'. The reported score can be multiplied by -1 to get the metric value.

Provided dataset contains following columns:
	target:           'price(원/kg)'
	past covariates:  ['supply(kg)']


In [ ]:
predictor.refit_full()

Refitting models via `refit_full` using all of the data (combined train and validation)...
	Models trained in this way will have the suffix '_FULL' and have NaN validation score.
	This process is not bound by time_limit, but should take less time than the original `fit` call.
Fitting model: Naive_FULL | Skipping fit via cloning parent ...
Fitting model: SeasonalNaive_FULL | Skipping fit via cloning parent ...
Fitting model: Theta_FULL | Skipping fit via cloning parent ...
Fitting model: AutoETS_FULL | Skipping fit via cloning parent ...
Fitting model: RecursiveTabular_FULL
	6.58    s     = Training runtime
Fitting model: DeepAR_FULL | Skipping fit via cloning parent ...
Fitting model: WeightedEnsemble_FULL | Skipping fit via cloning parent ...
Refit complete. Models trained: ['Naive_FULL', 'SeasonalNaive_FULL', 'Theta_FULL', 'AutoETS_FULL', 'RecursiveTabular_FULL', 'DeepAR_FULL', 'WeightedEnsemble_FULL']
Total runtime: 6.66 s
Updated best model to 'WeightedEnsemble_FULL' (Previously 'W

{'Naive': 'Naive_FULL',
 'SeasonalNaive': 'SeasonalNaive_FULL',
 'Theta': 'Theta_FULL',
 'AutoETS': 'AutoETS_FULL',
 'RecursiveTabular': 'RecursiveTabular_FULL',
 'DeepAR': 'DeepAR_FULL',
 'WeightedEnsemble': 'WeightedEnsemble_FULL'}

In [ ]:
# seed 고정
pred = predictor.predict(data, random_seed=42, )

INFO:lightning_fabric.utilities.seed:Global seed set to 42
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble_FULL


In [ ]:
pred

mean          0.1          0.2          0.3  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3426.412347  2349.498565  2726.841744  2994.646934   
        2023-03-05  3293.982080  1889.510259  2408.598467  2743.730616   
        2023-03-06  3172.800028  1485.670598  2073.364951  2482.318437   
        2023-03-07  3666.030306  1791.876063  2467.818536  2913.174549   
        2023-03-08  3443.615341  1377.039612  2103.272388  2595.799555   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   493.878759  -242.255973    55.929291   229.269553   
        2023-03-28   511.267609  -252.025734    52.193969   237.927427   
        2023-03-29   520.346587  -257.394225    45.520889   244.946413   
        2023-03-30   490.003327  -286.259576    -6.705123   206.803443   
        2023-03-31   481.447759  -312.998799    -7.888374   193.422306   

                            0.4          0.5          0.6          0.7  \
item_id timestamp                                                        
TG_A_J  2023-03-04  3228.578379  3430.265529  3645.112534  3863.204265   
        2023-03-05  3027.943811  3293.756760  3560.465503  3845.739610   
        2023-03-06  2831.988349  3163.171508  3495.209435  3834.548057   
        2023-03-07  3288.067169  3644.502834  4015.302795  4392.286792   
        2023-03-08  3014.349219  3433.558102  3838.323813  4259.420766   
...                         ...          ...          ...          ...   
RD_F_J  2023-03-27   373.790369   502.736502   631.718494   770.805151   
        2023-03-28   385.976983   523.989572   658.607733   800.636404   
        2023-03-29   388.139055   530.098638   663.465138   814.288995   
        2023-03-30   360.286462   499.672202   639.646128   788.032115   
        2023-03-31   352.587759   491.831899   638.289876   792.598603   

                            0.8          0.9  
item_id timestamp                             
TG_A_J  2023-03-04  4147.215314  4521.354744  
        2023-03-05  4182.490693  4662.820300  
        2023-03-06  4241.732594  4829.102900  
        2023-03-07  4843.676474  5485.213146  
        2023-03-08  4761.728773  5510.453466  
...                         ...          ...  
RD_F_J  2023-03-27   938.284033  1168.773244  
        2023-03-28   969.943636  1212.813434  
        2023-03-29   987.758677  1235.855457  
        2023-03-30   961.616129  1203.050251  
        2023-03-31   964.995259  1207.663636  

[1092 rows x 10 columns]

In [ ]:
submission = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/sample_submission.csv')

In [ ]:
submission['answer'] = pred.reset_index()['mean']
submission.loc[ submission['answer'] < 0.0, 'answer'] = 0.0
submission

,ID,answer
0,TG_A_J_20230304,3426.412347
1,TG_A_J_20230305,3293.982080
2,TG_A_J_20230306,3172.800028
3,TG_A_J_20230307,3666.030306
4,TG_A_J_20230308,3443.615341
...,...,...
1087,RD_F_J_20230327,493.878759
1088,RD_F_J_20230328,511.267609
1089,RD_F_J_20230329,520.346587
1090,RD_F_J_20230330,490.003327


In [ ]:
from datetime import datetime

# 'ID' 열에서 뒤 8글자를 추출하여 'date' 열을 만듭니다.
submission['date'] = pd.to_datetime(submission['ID'].str[-8:], format='%Y%m%d')

# 'date' 열에서 요일을 추출하여 'day_of_week' 열을 만듭니다. (0: 월요일, 1: 화요일, ..., 6: 일요일)
submission['day_of_week'] = submission['date'].dt.dayofweek

# 'day_of_week'이 6인 (일요일) 행들의 'answer' 값을 0으로 설정합니다.
submission.loc[submission['day_of_week'] == 6, 'answer'] = 0

# 'date' 및 'day_of_week' 열은 더 이상 필요하지 않으므로 제거합니다.
submission = submission.drop(['date', 'day_of_week'], axis=1)


In [ ]:
submission

,ID,answer
0,TG_A_J_20230304,3426.412347
1,TG_A_J_20230305,0.000000
2,TG_A_J_20230306,3172.800028
3,TG_A_J_20230307,3666.030306
4,TG_A_J_20230308,3443.615341
...,...,...
1087,RD_F_J_20230327,493.878759
1088,RD_F_J_20230328,511.267609
1089,RD_F_J_20230329,520.346587
1090,RD_F_J_20230330,490.003327


In [ ]:
submission.to_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/낭만더럽히기.csv', index=False)

#0 처리

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/train.csv')
trade = pd.read_csv('/content/drive/MyDrive/2023 IBA/데이콘_제주도/open/international_trade.csv')

In [ ]:
train

,ID,timestamp,item,corporation,location,supply(kg),price(원/kg)
0,TG_A_J_20190101,2019-01-01,TG,A,J,0.0,0.0
1,TG_A_J_20190102,2019-01-02,TG,A,J,0.0,0.0
2,TG_A_J_20190103,2019-01-03,TG,A,J,60601.0,1728.0
3,TG_A_J_20190104,2019-01-04,TG,A,J,25000.0,1408.0
4,TG_A_J_20190105,2019-01-05,TG,A,J,32352.0,1250.0
...,...,...,...,...,...,...,...
59392,RD_F_J_20230227,2023-02-27,RD,F,J,452440.0,468.0
59393,RD_F_J_20230228,2023-02-28,RD,F,J,421980.0,531.0
59394,RD_F_J_20230301,2023-03-01,RD,F,J,382980.0,574.0
59395,RD_F_J_20230302,2023-03-02,RD,F,J,477220.0,523.0


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 59397 entries, 0 to 59396
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           59397 non-null  object 
 1   timestamp    59397 non-null  object 
 2   item         59397 non-null  object 
 3   corporation  59397 non-null  object 
 4   location     59397 non-null  object 
 5   supply(kg)   59397 non-null  float64
 6   price(원/kg)  59397 non-null  float64
dtypes: float64(2), object(5)
memory usage: 3.2+ MB
